In [1]:
from code_app.obtener_precedentes import obtener_precedentes_completos
from code_app.filtrar_raices import filtrar_valores_raiz
from code_app.simular_impacto_raices import simular_impacto_raices
from code_app.filtrar_impactos import filtrar_mayores_impactos
from tabulate import tabulate  # pip install tabulate

In [2]:
ruta_excel = "/Users/alvarofelipepupuchemorales/Desktop/Proyecto Economia/assets/PALTA HASS  BCP.xlsx"
hoja_nombre = "Costos Agricolas 01 ha"
celda_objetivo = "E123"
# Conseguimos los precedentes
precedentes_totales = obtener_precedentes_completos(ruta_excel, hoja_nombre, celda_objetivo)
for p in precedentes_totales:
    print(p)

Costos Agricolas 01 ha!$E$45
Costos Agricolas 01 ha!$E$46
Costos Agricolas 01 ha!$E$48
Costos Agricolas 01 ha!$E$53
Costos Agricolas 01 ha!$E$55
Costos Agricolas 01 ha!$H$95
Costos Agricolas 01 ha!E11
Costos Agricolas 01 ha!E120
Costos Agricolas 01 ha!E29
Costos Agricolas 01 ha!E30
Costos Agricolas 01 ha!E31
Costos Agricolas 01 ha!E32
Costos Agricolas 01 ha!E33
Costos Agricolas 01 ha!E34
Costos Agricolas 01 ha!E41
Costos Agricolas 01 ha!E46
Costos Agricolas 01 ha!E50
Costos Agricolas 01 ha!E53
Costos Agricolas 01 ha!E84
Costos Agricolas 01 ha!E85
Costos Agricolas 01 ha!E86
Costos Agricolas 01 ha!E87
Costos Agricolas 01 ha!E88
Costos Agricolas 01 ha!E89
Costos Agricolas 01 ha!F11
Costos Agricolas 01 ha!F120
Costos Agricolas 01 ha!F15
Costos Agricolas 01 ha!F22
Costos Agricolas 01 ha!F23
Costos Agricolas 01 ha!F31
Costos Agricolas 01 ha!F41
Costos Agricolas 01 ha!F84
Costos Agricolas 01 ha!F85
Costos Agricolas 01 ha!F86
Costos Agricolas 01 ha!F87
Costos Agricolas 01 ha!F88
Costos Agricol

In [3]:
print("="*100)

raices = filtrar_valores_raiz(ruta_excel, precedentes_totales)

print("Raíces encontradas:")
for celda, valor in raices.items():
    print(f"{celda} → {valor}")

Raíces encontradas:
Costos Agricolas 01 ha!$E$45 → 25
Costos Agricolas 01 ha!$E$46 → 0.9
Costos Agricolas 01 ha!$E$53 → 1600
Costos Agricolas 01 ha!$H$95 → 0.07
Costos Agricolas 01 ha!E29 → 100
Costos Agricolas 01 ha!E46 → 0.9
Costos Agricolas 01 ha!E50 → 0.02
Costos Agricolas 01 ha!E53 → 1600
Costos Agricolas 01 ha!F98 → 30800
Costos Agricolas 01 ha!G15 → 350
Costos Agricolas 01 ha!G16 → 200
Costos Agricolas 01 ha!G17 → 1000
Costos Agricolas 01 ha!G19 → 121
Costos Agricolas 01 ha!G23 → 1200
Costos Agricolas 01 ha!G24 → 750
Costos Agricolas 01 ha!G25 → 200
Costos Agricolas 01 ha!G37 → 200
Costos Agricolas 01 ha!G57 → 0
Costos Agricolas 01 ha!G85 → 0
Costos Agricolas 01 ha!G88 → 3800
Costos Agricolas 01 ha!H95 → 0.07
Costos Agricolas 01 ha!J94 → 30800
Costos Agricolas 01 ha!V15 → 427
Costos Agricolas 01 ha!V17 → 1734
Costos Agricolas 01 ha!V19 → 480
Costos Agricolas 01 ha!V23 → 2200
Costos Agricolas 01 ha!V24 → 1200
Costos Agricolas 01 ha!V37 → 300
INSTALACION Y SIEMBRA!B19 → 900
INSTAL

In [4]:
import openpyxl
import re
from openpyxl.utils import column_index_from_string, get_column_letter

def generar_precedentes(ruta_excel, hoja_objetivo, celda_objetivo):
    """
    Genera un mapa de precedentes y fórmulas a partir de una celda objetivo en un Excel.

    Args:
        ruta_excel (str): La ruta al archivo Excel.
        hoja_objetivo (str): El nombre de la hoja en el Excel.
        celda_objetivo (str): La celda a partir de la cual se desea generar el mapa (ej. 'A1').

    Returns:
        tuple: Una tupla que contiene:
            - mapa (dict): Un diccionario donde las claves son referencias de celda (ej. 'A1')
              y los valores son funciones lambda de Python que representan la fórmula de esa celda.
              Cada lambda toma un argumento 'r' (una función de resolución para obtener valores de precedentes).
            - formulas (dict): Un diccionario donde las claves son referencias de celda y los valores
              son las fórmulas de Excel (limpias, sin '=') como cadenas.
            - raices (dict): Un diccionario donde las claves son referencias de celda y los valores
              son los valores numéricos de las celdas que no contienen fórmulas (las 'raíces' o entradas).
    """
    wb = openpyxl.load_workbook(ruta_excel, data_only=False)
    hoja = wb[hoja_objetivo]

    mapa = {}       # Almacenará las funciones lambda (fórmulas traducidas a Python)
    formulas = {}   # Almacenará las fórmulas de Excel (cadenas)
    raices = {}     # Almacenará los valores de las celdas sin fórmula (inputs)
    visitadas = set() # Para evitar bucles infinitos y reprocesamiento

    # Funciones auxiliares de Excel para simular en Python
    def npv(rate, values):
        # Asegura que 'values' sea un iterable numérico
        if not isinstance(values, (list, tuple, set)):
            if values is None: # Si no hay valores en el rango, NPV puede ser 0
                 return 0.0
            # Si es un solo valor numérico, conviértelo a lista
            try:
                values = [float(values)]
            except (ValueError, TypeError):
                raise TypeError(f"NPV values must be a list/tuple of numbers or a single number, received: {values}")

        return sum(v / (1 + rate) ** (i + 1) for i, v in enumerate(values))

    # Puedes añadir más funciones aquí si tus fórmulas las usan
    # Por ejemplo, para SUM, MAX, MIN, AVERAGE, etc.

    # Obtener fórmula de celda
    def obtener_formula(celda_ref):
        try:
            celda_obj = hoja[celda_ref]
            if celda_obj.data_type == 'f':
                return str(celda_obj.value) # Asegurarse de que es una cadena
            return None # No es una fórmula
        except KeyError:
            print(f"Advertencia: Celda {celda_ref} no encontrada en la hoja.")
            return None

    # Limpiar "=" y "+" inicial
    def limpiar_formula(formula_str):
        # Eliminar el '=' inicial si existe
        if formula_str.startswith('='):
            formula_str = formula_str[1:]
        # Eliminar el '+' inicial si existe (es común en Excel después del '=')
        if formula_str.startswith('+'):
            formula_str = formula_str[1:]
        return formula_str

    # Extraer referencias de celdas individuales y rangos
    def extraer_referencias_y_rangos(formula_str):
        # Primero, eliminar prefijos de hoja (ej. 'Hoja1'!)
        sin_hoja = re.sub(r"(?:'[^']+'|[A-Za-z0-9_]+)!", "", formula_str)
        
        # Encontrar todas las ocurrencias de referencias de celda A1 o rangos A1:B5
        # Esto captura las referencias de celda individuales o los rangos como una sola cadena
        references_and_ranges = re.findall(r'\b[A-Z]{1,3}[0-9]{1,7}(?::[A-Z]{1,3}[0-9]{1,7})?\b', sin_hoja)
        return references_and_ranges

    # Función recursiva para procesar celdas y sus precedentes
    def procesar_celda(celda_actual):
        if celda_actual in visitadas:
            return
        visitadas.add(celda_actual)

        excel_formula = obtener_formula(celda_actual)

        if not excel_formula:
            # Es una celda "raíz" (sin fórmula)
            try:
                # Intenta convertir el valor a float si es numérico, si no, déjalo como está
                valor_raiz = hoja[celda_actual].value
                raices[celda_actual] = float(valor_raiz) if isinstance(valor_raiz, (int, float)) else valor_raiz
            except (ValueError, TypeError):
                raices[celda_actual] = hoja[celda_actual].value
            return

        cleaned_formula = limpiar_formula(excel_formula)
        formulas[celda_actual] = cleaned_formula # Guardamos la fórmula limpia
        
        # Extraer referencias y rangos para procesamiento recursivo
        # Aquí necesitamos todas las celdas involucradas para procesarlas.
        refs_y_rangos = extraer_referencias_y_rangos(excel_formula)
        
        # Expandir los rangos para procesar individualmente cada celda que los compone.
        # Esto es importante para el `procesar_celda` recursivo.
        individual_precedents_for_recursion = set()
        for ref_or_range in refs_y_rangos:
            if ':' in ref_or_range:
                start_ref, end_ref = ref_or_range.split(':')
                # Obtener las coordenadas del rango
                start_col_letter = ''.join(filter(str.isalpha, start_ref))
                start_row = int(''.join(filter(str.isdigit, start_ref)))
                end_col_letter = ''.join(filter(str.isalpha, end_ref))
                end_row = int(''.join(filter(str.isdigit, end_ref)))

                start_col_idx = column_index_from_string(start_col_letter)
                end_col_idx = column_index_from_string(end_col_letter)

                for col_idx in range(start_col_idx, end_col_idx + 1):
                    for row_num in range(start_row, end_row + 1):
                        individual_precedents_for_recursion.add(f"{get_column_letter(col_idx)}{row_num}")
            else:
                individual_precedents_for_recursion.add(ref_or_range)
        
        # Procesar referencias recursivamente
        for ref in individual_precedents_for_recursion:
            procesar_celda(ref)

        # --- Construcción de la expresión Python para `eval()` ---
        python_formula_str = limpiar_formula(excel_formula)
        python_formula_str = re.sub(r"(?:'[^']+'|[A-Za-z0-9_]+)!", "", python_formula_str) # Quitar nombres de hoja
        
        # Primero, manejar funciones que toman rangos (ej. NPV, SUM)
        # Esto es un patrón común para funciones que operan en un conjunto de valores.
        # Definimos una función para reemplazar rangos por listas de llamadas a 'r()'
        def replace_excel_range_with_python_list(match):
            func_name = match.group(1) # Ej: NPV
            args_before_range = match.group(2) # Ej: 0.06,
            range_str = match.group(3) # Ej: G120:V120
            
            start_ref, end_ref = range_str.split(':')
            start_col_letter = ''.join(filter(str.isalpha, start_ref))
            start_row = int(''.join(filter(str.isdigit, start_ref)))
            end_col_letter = ''.join(filter(str.isalpha, end_ref))
            end_row = int(''.join(filter(str.isdigit, end_ref)))

            start_col_idx = column_index_from_string(start_col_letter)
            end_col_idx = column_index_from_string(end_col_letter)

            cells_in_range_list = []
            for col_idx in range(start_col_idx, end_col_idx + 1):
                for row_num in range(start_row, end_row + 1):
                    cells_in_range_list.append(f"r('{get_column_letter(col_idx)}{row_num}')")
            
            # El formato para la función en Python será: FUNCION(arg1, arg2, ..., [r('A1'), r('A2')])
            # Si no hay args_before_range, será solo la lista
            if args_before_range:
                return f"{func_name}({args_before_range.strip()}[{', '.join(cells_in_range_list)}])"
            else:
                return f"{func_name}([{', '.join(cells_in_range_list)}])"

        # Regex para funciones que toman rangos. Ajusta 'NPV|SUM|AVERAGE' a las funciones que uses.
        # Captura: 1. Nombre de la función, 2. Argumentos antes del rango (opcional), 3. El rango
        python_formula_str = re.sub(
            r'([A-Za-z_][A-Za-z0-9_]*)\(([^,)]*,\s*)?([A-Z]{1,3}[0-9]{1,7}:[A-Z]{1,3}[0-9]{1,7})\)',
            replace_excel_range_with_python_list,
            python_formula_str
        )
        
        # Segundo, reemplazar todas las referencias de celda individuales con r('CELDA')
        # Es crucial que esto se haga DESPUÉS de manejar los rangos,
        # para no reemplazar partes de los rangos antes de que se expandan.
        # Usamos el mismo patrón de regex para referencias individuales.
        all_individual_cell_refs = re.findall(r'\b[A-Z]{1,3}[0-9]{1,7}\b', python_formula_str)
        for ref in all_individual_cell_refs:
            # Asegurarse de que no estamos reemplazando partes ya manejadas en los rangos
            # Este es un punto delicado. Un enfoque más seguro es un parser AST.
            # Para la mayoría de los casos simples, esto funcionará.
            python_formula_str = re.sub(rf'\b{ref}\b', f"r('{ref}')", python_formula_str)

        try:
            # Mapear funciones de Excel a funciones Python
            # Aquí puedes agregar más funciones si las necesitas, ej: "ABS": abs, "ROUND": round
            eval_globals = {
                "npv": npv,
                "sum": sum,
                "max": max,
                "min": min,
                "abs": abs,
                # Excel "AND" se mapea a 'and' en Python, "OR" a 'or', pero en 'eval'
                # necesitas que sean funciones si se usan con sintaxis de función.
                # Para un "AND(A1,B1)" necesitarías def AND(a,b): return a and b
                # No soporta IF directamente sin una función IF(cond, val_true, val_false)
                "IF": lambda cond, true_val, false_val: true_val if cond else false_val
            }
            
            # Generar la función lambda para esta celda
            # La lambda toma un argumento 'r', que será una función que obtiene el valor de una celda
            mapa[celda_actual] = eval(f"lambda r: {python_formula_str}", eval_globals)
        except Exception as e:
            print(f"[ERROR] al evaluar fórmula en {celda_actual}: {python_formula_str}")
            print(f"       {e}")

    # Iniciar el procesamiento desde la celda objetivo
    procesar_celda(celda_objetivo)

    return mapa, formulas, raices

In [5]:
def evaluar_modelo(celda_objetivo, mapa_precedentes, valores_raices, hoja_actual_excel, workbook_excel_obj):
    """
    Evalúa el valor de la celda objetivo utilizando el mapa de precedentes y los valores de las raíces.

    Args:
        celda_objetivo (str): La celda cuyo valor se quiere calcular.
        mapa_precedentes (dict): El diccionario 'mapa' devuelto por generar_precedentes.
        valores_raices (dict): El diccionario 'raices' devuelto por generar_precedentes,
                               que puedes modificar para tu análisis de sensibilidad.
        hoja_actual_excel (str): El nombre de la hoja donde está la celda objetivo (para raíces no mapeadas)
        workbook_excel_obj (openpyxl.workbook.workbook.Workbook): El objeto workbook para leer directamente valores de celdas no mapeadas

    Returns:
        float or object: El valor calculado de la celda objetivo.
    """
    # Caché para almacenar los valores ya calculados de las celdas durante esta evaluación
    # Esto evita recalcular celdas múltiples veces si son precedentes de varias otras.
    cache_valores_calculados = {}
    
    # Esto es importante para manejar celdas que podrían no haber sido incluidas en el mapa
    # si no eran precedentes directos de la celda objetivo, pero podrían ser llamadas por r()
    # (por ejemplo, si el Excel es muy grande y solo extrajiste una rama pequeña).
    # O para obtener valores originales si no están en raices ni en formulas.
    # Es una función de fallback para r()
    def get_excel_cell_value_fallback(cell_ref):
        try:
            return workbook_excel_obj[hoja_actual_excel][cell_ref].value
        except Exception as e:
            print(f"Advertencia: No se pudo obtener el valor de {cell_ref} desde Excel: {e}")
            return None # O lanzar un error si prefieres que sea estricto

    def resolver_valor_celda(ref_celda):
        # 1. ¿Está en el caché de valores calculados?
        if ref_celda in cache_valores_calculados:
            return cache_valores_calculados[ref_celda]

        # 2. ¿Es una celda raíz (valor fijo)?
        if ref_celda in valores_raices:
            # Los valores de las raíces son la fuente de verdad inicial y las que variarás
            valor = valores_raices[ref_celda]
            cache_valores_calculados[ref_celda] = valor
            return valor

        # 3. ¿Es una celda con fórmula en nuestro mapa?
        if ref_celda in mapa_precedentes:
            # Si es una fórmula, la evaluamos recursivamente
            # La lambda toma 'resolver_valor_celda' como su 'r'
            valor = mapa_precedentes[ref_celda](resolver_valor_celda)
            cache_valores_calculados[ref_celda] = valor
            return valor
        
        # 4. Si no está en raíces ni en el mapa, intenta leer directamente del Excel original
        # Esto es un fallback importante para celdas no procesadas que puedan ser requeridas.
        valor = get_excel_cell_value_fallback(ref_celda)
        if isinstance(valor, (int, float)):
             cache_valores_calculados[ref_celda] = valor
             return valor
        
        # Si la celda no tiene valor numérico, podría ser texto o un error, lo cual causaría problemas.
        # Puedes decidir cómo manejar esto (ej. devolver 0, None, o lanzar un error)
        print(f"Advertencia: Celda {ref_celda} no es raíz, no tiene fórmula ni valor numérico directo. Su valor es: {valor}")
        return valor # Retorna el valor tal cual, puede ser None o str

    return resolver_valor_celda(celda_objetivo)

In [6]:
ruta_excel = "/Users/alvarofelipepupuchemorales/Desktop/Proyecto Economia/assets/PALTA HASS  BCP.xlsx"
hoja_nombre = "Costos Agricolas 01 ha"
celda_objetivo = "E123"

In [8]:
# --- Ejemplo de uso ---
if __name__ == "__main__":
    # Asegúrate de tener un archivo Excel de prueba, por ejemplo, 'mi_modelo.xlsx'
    # Con una hoja 'Hoja1' y algunas fórmulas.
    # Ejemplo:
    # A1: 100 (Raíz)
    # A2: 1.05 (Raíz)
    # B1: =A1 * A2 (Fórmula)
    # B2: =B1 + 50 (Fórmula)
    # C1: =NPV(0.05, A1:A3) (Fórmula con rango - necesitarás A3)
    # A3: 200 (Raíz para NPV)
    # D1: =+A1*B1 (Fórmula con +)
    # D2: =1*A1 (Fórmula con 1*)
    # E1: =IF(A1>50, B1, A2)

    ruta = ruta_excel # ¡CAMBIA ESTO!
    hoja_obj = hoja_nombre # ¡CAMBIA ESTO!
    celda_obj = celda_objetivo   # ¡CAMBIA ESTO a tu celda final!

    # --- Preparación: Generar el mapa ---
    # Necesitas el objeto workbook completo para el fallback en evaluar_modelo
    wb_full = openpyxl.load_workbook(ruta, data_only=False)
    
    print(f"Generando mapa de precedentes para {celda_obj}...")
    mapa_precedentes, formulas_extraidas, raices_originales = generar_precedentes(ruta, hoja_obj, celda_obj)
    
    print("\n--- Fórmulas Extraídas ---")
    for celda, formula_py in mapa_precedentes.items():
        print(f"Celda: {celda}, Fórmula Excel: {formulas_extraidas.get(celda, 'N/A')}, Función Python: {formula_py}")
    
    print("\n--- Raíces Originales ---")
    for celda, valor in raices_originales.items():
        print(f"Celda: {celda}, Valor: {valor}")

    # --- Análisis de Sensibilidad ---
    print(f"\n--- Iniciando Análisis de Sensibilidad para {celda_obj} ---")

    # 1. Calcular el valor base de la celda objetivo
    valor_base_objetivo = evaluar_modelo(celda_obj, mapa_precedentes, raices_originales.copy(), hoja_obj, wb_full)
    print(f"\nValor base de {celda_obj}: {valor_base_objetivo}")

    resultados_sensibilidad = {}



Generando mapa de precedentes para E123...
[ERROR] al evaluar fórmula en J95: r('J94')*$H$95
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en K95: r('K94')*$H$95
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en L95: r('L94')*$H$95
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en M95: r('M94')*$H$95
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en N95: r('N94')*$H$95
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en O95: r('O94')*$H$95
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en P95: r('P94')*$H$95
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en M45: 1*$E$45
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en M48: r('M45')*$E$48
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en M55: r('M48')*$E$55
       invalid syntax (<string>, line 1)
[ERROR] al evaluar fórmula en M46: r('M45')*$E$46
    